In [ ]:
import sys

import numpy as np
from aiida import load_profile
from tb_hamiltonian.workflows import compute_continuum_bands

In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters


In [ ]:
from tb_hamiltonian.continuum import create_Qn


use_mpi = False

L = 500

Qn = create_Qn(L)
b1, b2 = Qn[0], Qn[1]

model_params = dict(
    bond_length=1.425,
    interlayer_hopping=0.22,
    superlattice_potential_periodicity=500,
    superlattice_potential_amplitude=0.020,
    gate_bias=0.024,
    layer_potential_ratio=0.3,
    nearest_neighbor_order=3,
)

band_params = dict(
    high_sym_points={
        "Γ": np.array([0.0, 0.0]),
        "M": b1 / 2,
        "K": (b1 + b2) / 3,
    },
    path="Γ K M Γ",
    total_points=100,
    use_mpi=use_mpi,
)

# Metadata

For more scheduler options, see

https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/calculations/usage.html#options


In [ ]:
h, m, s = 1, 0, 0

NODES = 1
TASKS_PER_NODE = 1
CPUS_PER_TASK = 1

CONDA_ENV = ""
assert CONDA_ENV, "Please set the CONDA_ENV variable"


local_metadata = {
    "options": {
        "prepend_text": f"""
source $(conda info --base)/etc/profile.d/conda.sh && conda activate {CONDA_ENV}
        """,
        "withmpi": False,
        "resources": {
            "num_machines": NODES,
            "num_mpiprocs_per_machine": TASKS_PER_NODE,
            "num_cores_per_mpiproc": CPUS_PER_TASK,
        },
        "max_wallclock_seconds": h * 3600 + m * 60 + s,
    }
}

REMOTE_ENV_PATH = ""
assert REMOTE_ENV_PATH, \
    "Please set the REMOTE_ENV_PATH variable or comment out the assertion if running locally"

remote_metadata = {
    "options": {
        "prepend_text": f"""
module load cray-python
source {REMOTE_ENV_PATH}/bin/activate
        """,
        "withmpi": False,
        "resources": {
            "num_machines": NODES,
            "num_mpiprocs_per_machine": TASKS_PER_NODE,
            "num_cores_per_mpiproc": CPUS_PER_TASK,
        },
        "max_wallclock_seconds": h * 3600 + m * 60 + s,
    }
}

input_metadata = {
    "get_continuum_model": {
        "computer": "localhost",
        "metadata": local_metadata,
    },
    "get_continuum_band_structure": {
        "computer": "localhost",
        "metadata": local_metadata,
    },
}

# Build and submit work graph


In [ ]:
wg = compute_continuum_bands(
    **model_params,
    **band_params,
    metadata=input_metadata,
)

In [ ]:
wg.submit()

# Monitoring


In [ ]:
! verdi process status $wg.pk

# Analysis

In [ ]:
# from aiida.orm import load_node
# from tb_hamiltonian.utils import BandStructure

# wg_node = load_node(wg.pk)
# band_calculation = wg_node.called[-1]
# band_structure: BandStructure = band_calculation.outputs.result.value
# band_structure.plot(
#     fig_params={"ylim": (-0.04, 0.04)},
#     plot_params={"color": "k"},
# )